In [1]:
### Investment data
#Real investment kucoin investment data 
#Initialize investments.ipynb file in Resources folder.
# Prepare API calls for investment data collection.
# Pull investment data from first exchange.
# Organize investment data.
# Pull investment data from remaining exchanges.
# Organize investment data from remaining exchanges

In [3]:
import datetime
import time
import base64
import hmac
import hashlib
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import os


%matplotlib inline

# silence warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
dotenv_path = 'C:/Desktop/my_gitrepos/project-2/kucoin_data/k.env'
load_dotenv(dotenv_path)

True

In [5]:
api_key = os.getenv('KUCOIN_api_key')
api_secret = os.getenv('KUCOIN_api_secret')
api_passphrase = os.getenv('KUCOIN_api_passphrase')

In [6]:
def get_headers(url, method):
    now = int(time.time() * 1000)
    str_to_sign = str(now) + method + url
    signature = base64.b64encode(
        hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(
        hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
        "KC-API-SIGN": signature,
        "KC-API-TIMESTAMP": str(now),
        "KC-API-KEY": api_key,
        "KC-API-PASSPHRASE": passphrase,
        "KC-API-KEY-VERSION": "2"
    }
    return headers

def get_account_balance():
    url = '/api/v1/accounts'
    headers = get_headers(url, 'GET')
    response = requests.get('https://api.kucoin.com' + url, headers=headers)
    if response.status_code == 200:
        balances = response.json()['data']
        return balances
    else:
        print('Failed to get account balances. Error code:', response.status_code)
        return None

In [7]:
get_account_balance()

[{'id': '625a8e24c3c51b0001e1a13a',
  'currency': 'ROUTE',
  'type': 'trade',
  'balance': '3729.2422',
  'available': '3729.2422',
  'holds': '0'},
 {'id': '62d85e5b501cb3000163af50',
  'currency': 'ATOM',
  'type': 'trade',
  'balance': '50',
  'available': '50',
  'holds': '0'},
 {'id': '62d854f9cf97b00001868262',
  'currency': 'API3',
  'type': 'trade',
  'balance': '235',
  'available': '235',
  'holds': '0'},
 {'id': '62d8bdda975dca0001f55ac0',
  'currency': 'KNC',
  'type': 'trade',
  'balance': '142.316',
  'available': '142.316',
  'holds': '0'},
 {'id': '6259aad5a560990001c98ddb',
  'currency': 'USDT',
  'type': 'trade',
  'balance': '0.00007904',
  'available': '0.00007904',
  'holds': '0'},
 {'id': '62599d12c750d100018732c2',
  'currency': 'USDT',
  'type': 'main',
  'balance': '0',
  'available': '0',
  'holds': '0'}]

In [8]:
balances = get_account_balance()
df = pd.DataFrame(balances)
account_data= df[['currency', 'balance', 'available','holds']]
account_data.head()

,currency,balance,available,holds
0,ROUTE,3729.2422,3729.2422,0
1,ATOM,50,50,0
2,API3,235,235,0
3,KNC,142.316,142.316,0
4,USDT,0.00007904,0.00007904,0


In [9]:
account_data.head()

,currency,balance,available,holds
0,ROUTE,3729.2422,3729.2422,0
1,ATOM,50,50,0
2,API3,235,235,0
3,KNC,142.316,142.316,0
4,USDT,0.00007904,0.00007904,0


In [10]:
def get_ticker(symbol):
    url = f'/api/v1/market/orderbook/level1?symbol={symbol}'
    headers = get_headers(url, 'GET')
    response = requests.get('https://api.kucoin.com' + url, headers=headers)
    if response.status_code == 200:
        ticker_data = response.json()
        return ticker_data['data']
    else:
        print('Failed to get ticker data for symbol', symbol, '. Error code:', response.status_code)
        return None

In [11]:
get_ticker('KNC-USDT')

{'time': 1685013133126,
 'sequence': '197889995',
 'price': '0.5938',
 'size': '0.1817',
 'bestBid': '0.5977',
 'bestBidSize': '333',
 'bestAsk': '0.5994',
 'bestAskSize': '4900.0384'}

In [12]:
def get_all_tickers():
    url = '/api/v1/market/allTickers'
    headers = get_headers(url, 'GET')
    response = requests.get('https://api.kucoin.com' + url, headers=headers)
    if response.status_code == 200:
        ticker_data = json.loads(response.text)
        return ticker_data
    else:
        print('Failed to get ticker data. Error code:', response.status_code)

In [13]:
#get_all_tickers()

In [14]:
def get_symbols():
    url = '/api/v2/symbols'
    headers = get_headers(url, 'GET')
    response = requests.request('get', 'https://api.kucoin.com'+url, headers=headers)
    if response.status_code == 200:
        symbols_data = response.json()['data']
        symbols_df = pd.DataFrame(symbols_data)
        return symbols_df
    else:
        print('Failed to get symbols data. Error code:', response.status_code)       

In [15]:
get_symbols()

,symbol,name,baseCurrency,quoteCurrency,feeCurrency,market,baseMinSize,quoteMinSize,baseMaxSize,quoteMaxSize,baseIncrement,quoteIncrement,priceIncrement,priceLimitRate,minFunds,isMarginEnabled,enableTrading
0,LOKI-BTC,OXEN-BTC,LOKI,BTC,BTC,BTC,0.1,0.00001,10000000000,99999999,0.0001,0.000000001,0.000000001,0.1,0.000001,False,True
1,LOKI-ETH,OXEN-ETH,LOKI,ETH,ETH,ALTS,0.1,0.0001,10000000000,99999999,0.0001,0.0000001,0.0000001,0.1,0.00001,False,True
2,NRG-BTC,NRG-BTC,NRG,BTC,BTC,BTC,0.01,0.00001,10000000000,99999999,0.0001,0.00000001,0.00000001,0.1,0.000001,False,True
3,AVA-USDT,AVA-USDT,AVA,USDT,USDT,USDS,0.1,0.01,10000000000,99999999,0.0001,0.000001,0.000001,0.1,0.1,False,True
4,FET-BTC,FET-BTC,FET,BTC,BTC,BTC,0.1,0.00001,10000000000,99999999,0.0001,0.000000001,0.000000001,0.1,0.000001,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,BOB-USDT,BOB-USDT,BOB,USDT,USDT,USDS,10000,0.1,10000000000,99999999,0.0001,0.00000001,0.00000001,0.1,0.1,False,True
1344,LADYS-USDT,LADYS-USDT,LADYS,USDT,USDT,USDS,10000000,0.1,1000000000000,99999999,0.00001,0.00000000001,0.00000000001,0.1,0.1,False,True
1345,RFD-USDT,RFD-USDT,RFD,USDT,USDT,USDS,10000,0.1,10000000000,99999999,0.0001,0.00000001,0.00000001,0.1,0.1,False,True
1346,TENET-USDT,TENET-USDT,TENET,USDT,USDT,USDS,10,0.1,10000000000,99999999,0.0001,0.00001,0.00001,0.1,0.1,False,True


In [16]:
def clean_symbols_data(symbols_df):
    symbols_df = symbols_df.loc[:, ['symbol', 'name', 'baseCurrency', 'quoteCurrency', 'feeCurrency', 'market']]
    symbols_df = symbols_df.rename(columns={
        'baseCurrency': 'base_currency',
        'quoteCurrency': 'quote_currency',
        'feeCurrency': 'fee_currency',
    })
    symbols_df = symbols_df.set_index('symbol')
    return symbols_df
symbols_df = get_symbols()
symbols_cleaned_df = clean_symbols_data(symbols_df)
symbols_cleaned_df.head()
symbols_cleaned_df.tail()

,name,base_currency,quote_currency,fee_currency,market
symbol,,,,,
BOB-USDT,BOB-USDT,BOB,USDT,USDT,USDS
LADYS-USDT,LADYS-USDT,LADYS,USDT,USDT,USDS
RFD-USDT,RFD-USDT,RFD,USDT,USDT,USDS
TENET-USDT,TENET-USDT,TENET,USDT,USDT,USDS
HIBAKC-USDT,HIBAKC-USDT,HIBAKC,USDT,USDT,USDS


In [17]:
def get_symbols_pair():
    symbols_df = get_symbols()
    return symbols_df['symbol']

In [18]:
get_symbols_pair()

0          LOKI-BTC
1          LOKI-ETH
2           NRG-BTC
3          AVA-USDT
4           FET-BTC
           ...     
1343       BOB-USDT
1344     LADYS-USDT
1345       RFD-USDT
1346     TENET-USDT
1347    HIBAKC-USDT
Name: symbol, Length: 1348, dtype: object

In [19]:
def get_base_currency():
    symbols_df = get_symbols()
    return symbols_df['baseCurrency']

In [20]:
get_base_currency()

0         LOKI
1         LOKI
2          NRG
3          AVA
4          FET
         ...  
1343       BOB
1344     LADYS
1345       RFD
1346     TENET
1347    HIBAKC
Name: baseCurrency, Length: 1348, dtype: object

In [21]:
def get_quote_currency():
    symbols_df = get_symbols()
    return symbols_df['quoteCurrency']

In [22]:
get_quote_currency()

0        BTC
1        ETH
2        BTC
3       USDT
4        BTC
        ... 
1343    USDT
1344    USDT
1345    USDT
1346    USDT
1347    USDT
Name: quoteCurrency, Length: 1348, dtype: object

In [23]:
def get_deposit_list(currency=None, page=None, pageSize=None):
    url = '/api/v1/deposits'
    headers = get_headers(url, 'GET')
    params = {}
    if currency:
        params['currency'] = currency
    if page:
        params['page'] = page
    if pageSize:
        params['pageSize'] = pageSize
    
    response = requests.get('https://api.kucoin.com' + url, headers=headers, params=params)
    if response.status_code == 200:
        deposit_list = response.json()
        return deposit_list
    else:
        print('Failed to get deposit list. Error code:', response.status_code)

In [24]:
get_deposit_list()

{'code': '200000',
 'data': {'currentPage': 1,
  'pageSize': 50,
  'totalNum': 2,
  'totalPage': 1,
  'items': [{'currency': 'USDT',
    'chain': 'eth',
    'status': 'SUCCESS',
    'address': '0x6771e5ff1bd0e575efa15464b12dc0956b4803ec',
    'memo': '',
    'isInner': False,
    'amount': '1145.81000000',
    'fee': '0.00000000',
    'walletTxId': '0x5c3dd119172a19303d4dea59259a6a00d40e26b6a97c55986cde1ce616f555c4@1e998a0ab8a81db064bd0780da57c071',
    'createdAt': 1656513999000,
    'updatedAt': 1656514119000,
    'remark': 'Deposit',
    'arrears': False},
   {'currency': 'USDT',
    'chain': 'trx',
    'status': 'SUCCESS',
    'address': 'TZ88wBT8mZcD88q7o3dxuLYsvM7tfz21SU',
    'memo': '',
    'isInner': False,
    'amount': '13078.45270000',
    'fee': '0.00000000',
    'walletTxId': 'fef93e1a0cc9998eb87530a0eadc6d2d22de2859ba2f6a928646b9277dc2a1c4@9f03b5d10da04073eb4fdf99e5ad0a12',
    'createdAt': 1650040082000,
    'updatedAt': 1650040082000,
    'remark': 'Deposit',
    'arre

In [25]:
def get_base_fee():
    url = '/api/v1/base-fee'
    headers = get_headers(url, 'GET')
    response = requests.get('https://api.kucoin.com'+url, headers=headers)
    if response.status_code == 200:
        fees_data = response.json()['data']
        fees_df = pd.DataFrame(fees_data, index=[0])
        return fees_df
    else:
        print('Failed to get trade fees data. Error code:', response.status_code)

In [26]:
get_base_fee()

,takerFeeRate,makerFeeRate
0,0.001,0.001


In [27]:
def get_actual_fee_rate(symbol):
    url = '/api/v1/trade-fees'
    headers = get_headers(url, 'GET')
    response = requests.get('https://api.kucoin.com'+url+'?symbols='+symbol, headers=headers)
    if response.status_code == 200:
        fees_data = response.json()['data']
        fees_df = pd.DataFrame(fees_data)
        return fees_df
    else:
        print('Failed to get actual fee rate data. Error code:', response.status_code)

In [28]:
get_actual_fee_rate('KNC-USDT')

Failed to get actual fee rate data. Error code: 401


In [29]:
def get_24hr_stats(symbol):
    url = 'https://api.kucoin.com/api/v1/market/stats'
    params = {'symbol': symbol}
    response = requests.get(url, params=params)
    data = json.loads(response.text)['data']
    return data

In [30]:
get_24hr_stats('KNC-USDT')

{'time': 1685019200619,
 'symbol': 'KNC-USDT',
 'buy': '0.5977',
 'sell': '0.5994',
 'changeRate': '-0.0287',
 'changePrice': '-0.0176',
 'high': '0.6119',
 'low': '0.5824',
 'vol': '10832.2635',
 'volValue': '6452.61904874',
 'last': '0.5938',
 'averagePrice': '0.61221686',
 'takerFeeRate': '0.001',
 'makerFeeRate': '0.001',
 'takerCoefficient': '1',
 'makerCoefficient': '1'}

In [31]:
def get_market_list():
    endpoint = 'https://api.kucoin.com/api/v1/markets'
    response = requests.get(endpoint)
    data = json.loads(response.content)
    markets = data['data']
    print(markets)

In [32]:
get_market_list()

['USDS', 'BTC', 'ALTS', 'ETF', 'KCS', 'Meme', 'Hong-Kong', 'FIAT', 'Arbitrum', 'DeFi', 'Shanghai-Upgrade', 'NFT', 'AI', 'Polkadot', 'Layer 1', 'NFT-ETF']


In [33]:
def get_trade_histories(symbol):
    url = 'https://api.kucoin.com/api/v1/market/histories?symbol={}'.format(symbol)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['data']
    else:
        return None

In [52]:
#get_trade_histories('KNC-BTC')

In [35]:
def get_currencies():
    url = '/api/v1/currencies'
    response = requests.get('https://api.kucoin.com' + url)
    if response.status_code == 200:
        currencies_data = response.json()
        return currencies_data
    else:
        print('Failed to get currencies data. Error code:', response.status_code)

In [36]:
#get_currencies()

In [37]:
def get_fiat_price(symbol):
    url = 'https://api.kucoin.com/api/v1/prices'
    params = {"symbols": symbol}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()['data']
        return float(data[symbol])
    else:
        print(f"Failed to get {symbol} price. Error code:", response.status_code)

In [38]:
get_fiat_price('USDT')

0.9998

In [53]:
def get_klines(symbol, interval='1day'):
    url = f'https://api.kucoin.com/api/v1/market/candles?type={interval}&symbol={symbol}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()['data']
        klines_data = pd.DataFrame(data, columns=['time', 'open', 'close', 'high', 'low', 'volume', 'turnover'])
        klines_data['time'] = pd.to_datetime(klines_data['time'], unit='ms')
        klines_data.set_index('time', inplace=True)
        klines_data = klines_data.astype(float)
        return klines_data
    else:
        print(f'Failed to get {symbol} Klines data. Error code:', response.status_code)
        return None

In [54]:
symbol = 'BTC-USDT'
interval = '1day'
klines_data = get_klines(symbol, interval)
klines_data.head()

,open,close,high,low,volume,turnover
time,,,,,,
1970-01-20 12:02:52.800,26328.3,26302.4,26489.9,25871.2,2280.091418,5.980918e+07
1970-01-20 12:01:26.400,27219.4,26328.2,27219.4,26086.0,3329.016103,8.834830e+07
1970-01-20 12:00:00.000,26846.1,27219.4,27480.0,26800.0,3295.821547,8.976906e+07
1970-01-20 11:58:33.600,26744.7,26846.1,27095.1,26539.5,2117.441439,5.680578e+07
1970-01-20 11:57:07.200,27104.8,26744.7,27279.4,26655.2,1550.885030,4.177792e+07


In [41]:
def get_investment_data(url='/api/v1/accounts'):
    headers = get_headers(url, 'GET')
    response = requests.get(f'https://api.kucoin.com{url}', headers=headers)
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data['data'])
    else:
        print(f"Failed to get investment data. Error code: {response.status_code}")
        return None

In [42]:
get_investment_data()

,id,currency,type,balance,available,holds
0,625a8e24c3c51b0001e1a13a,ROUTE,trade,3729.2422,3729.2422,0
1,62d85e5b501cb3000163af50,ATOM,trade,50,50,0
2,62d854f9cf97b00001868262,API3,trade,235,235,0
3,62d8bdda975dca0001f55ac0,KNC,trade,142.316,142.316,0
4,6259aad5a560990001c98ddb,USDT,trade,0.00007904,0.00007904,0
5,62599d12c750d100018732c2,USDT,main,0,0,0


In [43]:
def get_assets_data():
    tickers = get_all_tickers()['data']['ticker']
    balances = get_account_balance()
    assets_data = {}
    for row in balances:
        currency = row['currency']
        holdings = row['balance']
        ticker = next((item for item in tickers if item['symbol'] == f"{currency}-USDT"), None)
        if ticker is not None:
            price = float(ticker['last'])
            value = price * float(holdings)
            asset_data = {'symbol': currency, 'holdings': holdings, 'price': price, 'value': value}
            assets_data[currency] = asset_data
    return assets_data

In [44]:
get_assets_data()

{'ROUTE': {'symbol': 'ROUTE',
  'holdings': '3729.2422',
  'price': 2.5533,
  'value': 9521.874109260001},
 'ATOM': {'symbol': 'ATOM',
  'holdings': '50',
  'price': 10.5001,
  'value': 525.005},
 'API3': {'symbol': 'API3',
  'holdings': '235',
  'price': 1.176,
  'value': 276.35999999999996},
 'KNC': {'symbol': 'KNC',
  'holdings': '142.316',
  'price': 0.5938,
  'value': 84.5072408}}

In [45]:
def get_ohlc_data(symbol, interval='1day', columns=['timestamp', 'open', 'close', 'high', 'low', 'volume', 'turnover']):
    url = f'https://api.kucoin.com/api/v1/market/candles?symbol={symbol}&type={interval}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()['data']
        ohlc_data = []
        for candle in data:
            ohlc_data.append({
                columns[0]: candle[0],
                columns[1]: float(candle[1]),
                columns[2]: float(candle[4]),
                columns[3]: float(candle[2]),
                columns[4]: float(candle[3]),
                columns[5]: float(candle[5]),
                columns[6]: float(candle[6])
            })
        return ohlc_data
    else:
        print(f'Failed to get {symbol} OHLC data. Error code:', response.status_code)

In [46]:
#get_ohlc_data('KNC-USDT', '1day')

In [47]:
def get_total_portfolio_value():
    tickers = get_all_tickers()['data']['ticker']
    balances = get_account_balance()
    total_value = 0.0
    for row in balances:
        currency = row['currency']
        holdings = row['balance']
        ticker = next((item for item in tickers if item['symbol'] == f"{currency}-USDT"), None)
        if ticker is not None:
            price = float(ticker['last'])
            value = price * float(holdings)
            total_value += value
    return total_value

In [48]:
get_total_portfolio_value()

10407.74635006

In [49]:
def get_total_profit_loss():
    balances = get_account_balance()
    total_profit_loss = 0.0
    for balance in balances:
        currency = balance['currency']
        holdings = float(balance['balance'])
        if holdings == 0:
            continue
        ticker = next((item for item in get_all_tickers()['data']['ticker'] if item['symbol'] == f"{currency}-USDT"), None)
        if ticker:
            price = float(ticker.get('last', 0))
            avg_price = float(balance.get('avgPrice', 0))
            profit_loss = (price - avg_price) * holdings
            total_profit_loss += profit_loss
    return total_profit_loss

In [50]:
get_total_profit_loss()

10407.74635006